In [ ]:
import os
os.environ["MKL_THREADING_LAYER"] = "AMD"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["TORCH_DEVICE"] = "cuda"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

import itertools
from attrs import define
from codes.train import train
# from codes.train_dist import train
from codes.optimizers.base import Optimizer
# from code.problems import Problem
from codes import Loss
from codes.datasets import Dataset
from codes.models import Model
from codes.attacks import Attack

%load_ext autoreload
%autoreload 2

In [ ]:
def zip_dict(**kwargs):
    keys = kwargs.keys()
    for instance in zip(*kwargs.values()):
        yield dict(zip(keys, instance))

def product_dict(**kwargs):
    keys = kwargs.keys()
    for instance in itertools.product(*kwargs.values()):
        yield dict(zip(keys, instance))

In [ ]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_CHECK_EXIST'] = 'False'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

# 4 classes CIFAR10

In [ ]:
@define
class BaseConfig():
    nepochs:         int = 40
    npeers:          int = 40
    seed:            int = 0

    loss:           Loss = Loss.CrossEntropyLoss
    model:         Model = Model.ResNet18
    dataset:     Dataset = Dataset.CIFAR10

    # nsamples:        int = 1000
    valnsamples_:     int = 300
    valenabled_:     bool = False
    nclasses:        int = 3
    hratio:        float = None

    optimizer: Optimizer = None
    batchsize:       int = 75
    lr:            float = 1e-2
    attack:       Attack = None

    trueweights:    bool = None

    mdbatchsize_:    int = None
    mdniters_:       int = None
    mdlr_:           int = None
    mdfull_:        bool = None

args_grid = dict(
    # hratio=[0.99, 0.9, 0.7, 0.5],
    hratio=[0.5],
    # hratio=[0.99],
    mdlr_=[0.1],
    seed=[0],
    # seed=[0,1,2,3,4,5,6],
    valenabled_=[False],
    # valenabled_=[True, False]
)

os.environ['MLFLOW_RUN_TAGS'] = str(dict(about=f'base'))

for d in product_dict(**args_grid):
    
    os.environ['MLFLOW_RUN_NAME'] = 'SGD Full'
    config = BaseConfig(**d)
    config.mdlr_ = None
    config.optimizer = Optimizer.SGD
    config.trueweights = False
    %time train(config)
    
    # os.environ['MLFLOW_RUN_NAME'] = 'SGD Ideal'
    # config = BaseConfig(**d)
    # config.mdlr_ = None
    # config.hratio = None
    # config.optimizer = Optimizer.SGD
    # config.trueweights = True
    # %time train(config)

    # os.environ['MLFLOW_RUN_NAME'] = 'MeritFed MD'
    # config = BaseConfig(**d)
    # config.optimizer = Optimizer.MeritFed
    # config.mdfull_ = True
    # config.mdniters_ = 1
    # %time train(config)

    # os.environ['MLFLOW_RUN_NAME'] = 'MeritFed SMD'
    # config = BaseConfig(**d)
    # config.optimizer = Optimizer.MeritFed
    # config.mdfull_ = False
    # config.mdbatchsize_ = 90
    # config.mdniters_ = 10
    # %time train(config)
    
    # config = BaseConfig(**d)
    # config.optimizer = Optimizer.TAWT
    # os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    # config.mdlr_ = None
    # %time train(config)

    # config = BaseConfig(**d)
    # config.optimizer = Optimizer.FedAdp
    # os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    # config.mdlr_ = None
    # %time train(config)

# fedfomo

In [ ]:
@define
class BaseConfig():
    nepochs:         int = 40
    npeers:          int = 20 # or 40
    seed:            int = 0

    loss:           Loss = Loss.CrossEntropyLoss
    model:         Model = Model.ResNet18
    dataset:     Dataset = Dataset.CIFAR10

    # nsamples:        int = 1000
    valnsamples_:     int = 300
    valenabled_:     bool = False
    nclasses:        int = 3
    hratio:        float = None

    optimizer: Optimizer = None
    batchsize:       int = 75
    lr:            float = 1e-2
    attack:       Attack = None

    trueweights:    bool = None

    mdbatchsize_:    int = None
    mdniters_:       int = None
    mdlr_:           int = None
    mdfull_:        bool = None

args_grid = dict(
    # hratio=[0.99, 0.9, 0.7, 0.5],
    mdlr_=[0.1],
    seed=[0,1,2,3,4,5,6],
    valenabled_=[False],
)

os.environ['MLFLOW_RUN_TAGS'] = str(dict(about=f'fedfomo'))

for d in product_dict(**args_grid):

    config = BaseConfig(**d)
    config.optimizer = Optimizer.FedFomo
    os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    config.mdlr_ = None
    config.mdbatchsize_ = 75
    %time train(config)